# Projekt 1 - Nimby - Paweł Malisz, Mikołaj Białek - Ocena: 4.0

## 1. Cel ćwiczenia

Celem ćwiczenia jest modyfikacja zaproponowanej w bibliotece easyAI implementacji gry Nim, tak aby zawierała element losowy. Losowość polegać będzie na tym, że przy wykonywaniu ruchu jest 10. procentowe prawdopodobieństwo, że gracz bierze ze stosu o 1 element mniej niż zamierzał. Zmodyfikowana gra zostanie wielokrotnie uruchomiona (w wersjach deterministycznej i probabilistycznej) z dwoma graczami AI, dla dwóch różnych głębokości, a otrzymane wyniki (ilość wygranych, przegranych, remisów) porównane. Następnie gra zostanie wielokrotnie uruchomiona dla dwóch różnych algorytmów (Negamax z i bez odcięcia alfa-beta) z dwoma ustawieniami maksymalnej głębokości. Wyniki zostaną porównane, a dodatkowo policzony zostanie średni czas spędzony na wybraniu akcji przez każdy wariant AI.

## 2. Opis problemu

Nim to gra dla dwóch osób z użyciem pinonków, które dzieli się na kupki dowolnej wielkości. Następnie gracze zabierają na zmianę dowolną, niezerową liczbę pionków. W jednym ruchu można zebrać tylko z jednej kupki. Wygrywa gracz, który zabiera ostatni pionek.

Przez naturę samej gry, jeśli obaj gracze grają optymalnie, można z góry stwierdzić, który z nich wygra. W takim razie więc, jeśli dla zadanej planszy, 2 algorytmy AI będą grały optymalnie zawsze wygra ten sam. Czy tak się realnie stanie zobaczymy później, po wykonaniu testów. Dodanie element losowego powinno zatem zmienić wyniki, w stosunku do wariantu deterministycznego gry. Testy będziemy przeprowadzać dla stałej planszy, składającej się z 4 kupek pionków o odpowiednich ilościach: 2, 3, 4, 5.

## 3. Realizacja rozwiązania

### Kod gry Nim z https://github.com/Zulko/easyAI
Usunęliśmy funkcję `unmake_move` (funkcja ta pszyspiesza nieco działanie algorytmu, ale kiedy jest ona obecna niżej zaprezentowana przez nas wersja probabilistyczna gry nie działa) oraz dodaliśmy do konstruktora parametr `current_player`, tak aby dało się zmieniać gracza, rozpoczynającego rozgrywkę.

In [2]:
from easyAI import TwoPlayerGame


class Nim(TwoPlayerGame):
    """
    The game starts with 4 piles of 5 pieces. In turn the players
    remove as much pieces as they want, but from one pile only. The
    player that removes the last piece loses.
    Parameters
    ----------
    players
      List of the two players e.g. [HumanPlayer(), HumanPlayer()]
    piles:
      The piles the game starts with. With piles=[2,3,4,4] the
      game will start with 1 pile of 2 pieces, 1 pile of 3 pieces, and 2
      piles of 4 pieces.
    max_removals_per_turn
      Max number of pieces you can remove in a turn. Default is no limit.
    """

    def __init__(self, players=None, max_removals_per_turn=None, piles=(2, 3, 4, 5), current_player=1):
        """ Default for `piles` is 5 piles of 5 pieces. """
        self.players = players
        self.piles = list(piles)
        self.max_removals_per_turn = max_removals_per_turn
        self.current_player = current_player

    def possible_moves(self):
        return [
            "%d,%d" % (i + 1, j)
            for i in range(len(self.piles))
            for j in range(
                1,
                self.piles[i] + 1
                if self.max_removals_per_turn is None
                else min(self.piles[i] + 1, self.max_removals_per_turn),
            )
        ]

    def make_move(self, move):
        move = list(map(int, move.split(",")))
        self.piles[move[0] - 1] -= move[1]

    def show(self):
        print(" ".join(map(str, self.piles)))

    def win(self):
        return max(self.piles) == 0

    def is_over(self):
        return self.win()

    def scoring(self):
        return 100 if self.win() else 0

    def ttentry(self):
        return tuple(self.piles)  # optional, speeds up AI

### Wersja gry z 10% szansą wzięcia 1 elementu mniej niż zamierzano

In [3]:
import random

class MissNim(Nim):
    def make_move(self, move):
        move = list(map(int, move.split(",")))
        m = move[1] - 1 if random.randint(1, 10) == 1 else move[1]
        self.piles[move[0] - 1] -= m

### Rozegranie gry

Funkcja pomocnicza odpowiadająca za uruchamienie daną ilość razy (`how_many_times`), danej gry (`game_type`), z danym algorytmem AI (`ai_algorithm`), daną głębokością (`depth`) oraz odcięciem alpha-beta (`alpha_beta`). Co wykonanie zmienia gracza rozpoczynającego.

In [1]:
from easyAI import AI_Player, Negamax
import time
from statistics import mean

def play_nim(how_many_times, game_type, ai_algorithm, depth, alpha_beta = None):
    ai = get_ai(ai_algorithm, depth, alpha_beta)
    score_board = [0, 0]
    times = []
    for i in range(how_many_times):
        start = time.time()
        game = game_type([AI_Player(ai), AI_Player(ai)], current_player=i%2+1)
        game.play()
        score_board[game.current_player - 1] += 1
        times.append((time.time() - start) / game.nmove) # devide the time by the number of moves
        print("player %d wins" % game.current_player)
    return [get_text_by_game_type(game_type), depth, get_text_by_alpha_beta(alpha_beta), how_many_times, score_board[0], score_board[1], mean(times)]

def get_ai(ai_algorithm, depth, alpha_beta):
    return ai_algorithm(depth) if alpha_beta is None else ai_algorithm(depth, win_score = alpha_beta)

def get_text_by_game_type(game_type):
    if game_type == Nim:
        return "Deterministic"
    if game_type == MissNim:
        return "Probabilistic"
    return "Unknown"

def get_text_by_alpha_beta(alpha_beta):
    return "-" if alpha_beta is None else alpha_beta

1. Najpierw uruchamiamy grę 40 razy w wersji deterministycznej oraz 40 razy w wersji probabilistycznej z algorytmem Negamax (głębokość ustawiona na 2, bez odcięcia). 
2. Następnie wykonujemy to samo, lecz z odciągiem alpha równym 3.
3. Na końcu wykonujemy krok 1 i 2 ustawiając głębokość na 5.

In [33]:
from tabulate import tabulate

records = []
records.append(play_nim(40, Nim, Negamax, 2))
records.append(play_nim(40, MissNim, Negamax, 2))
records.append(play_nim(40, Nim, Negamax, 2, 3))
records.append(play_nim(40, MissNim, Negamax, 2, 3))
records.append(play_nim(40, Nim, Negamax, 5))
records.append(play_nim(40, MissNim, Negamax, 5))
records.append(play_nim(40, Nim, Negamax, 5, 3))
records.append(play_nim(40, MissNim, Negamax, 5, 3))

2 3 4 5

Move #1: player 1 plays 1,1 :
1 3 4 5

Move #2: player 2 plays 1,1 :
0 3 4 5

Move #3: player 1 plays 2,1 :
0 2 4 5

Move #4: player 2 plays 2,1 :
0 1 4 5

Move #5: player 1 plays 2,1 :
0 0 4 5

Move #6: player 2 plays 3,1 :
0 0 3 5

Move #7: player 1 plays 3,1 :
0 0 2 5

Move #8: player 2 plays 3,1 :
0 0 1 5

Move #9: player 1 plays 4,5 :
0 0 1 0

Move #10: player 2 plays 3,1 :
0 0 0 0
player 1 wins
2 3 4 5

Move #1: player 2 plays 1,1 :
1 3 4 5

Move #2: player 1 plays 1,1 :
0 3 4 5

Move #3: player 2 plays 2,1 :
0 2 4 5

Move #4: player 1 plays 2,1 :
0 1 4 5

Move #5: player 2 plays 2,1 :
0 0 4 5

Move #6: player 1 plays 3,1 :
0 0 3 5

Move #7: player 2 plays 3,1 :
0 0 2 5

Move #8: player 1 plays 3,1 :
0 0 1 5

Move #9: player 2 plays 4,5 :
0 0 1 0

Move #10: player 1 plays 3,1 :
0 0 0 0
player 2 wins
2 3 4 5

Move #1: player 1 plays 1,1 :
1 3 4 5

Move #2: player 2 plays 1,1 :
0 3 4 5

Move #3: player 1 plays 2,1 :
0 2 4 5

Move #4: player 2 plays 2,1 :
0 1 4 5

Move #5: 

## 5. Rezultat

Wyświetlamy tabelę z wynikami dla poszczególnych graczy.

In [34]:
print(tabulate(records, headers = ['Type', 'Depth', 'Alpha-Beta Cutoff', 'Attempts', 'Player 1 wins', 'Player 2 wins', 'Average time per one move'], tablefmt='orgtbl'))

| Type          |   Depth | Alpha Cutoff   |   Attempts |   Player 1 wins |   Player 2 wins |   Average time per one move |
|---------------+---------+----------------+------------+-----------------+-----------------+-----------------------------|
| Deterministic |       2 | -              |         40 |              20 |              20 |                 0.00111519  |
| Probabilistic |       2 | -              |         40 |              21 |              19 |                 0.00100209  |
| Deterministic |       2 | 3              |         40 |              20 |              20 |                 0.000917502 |
| Probabilistic |       2 | 3              |         40 |              22 |              18 |                 0.000983497 |
| Deterministic |       5 | -              |         40 |              20 |              20 |                 0.0196983   |
| Probabilistic |       5 | -              |         40 |              18 |              22 |                 0.025891    |
| Determ

### Wnioski

Pierwsze wnioski, które się nasuwają są takie, że w przypadku gier deterministycznych (bez udziału czynnika losowego) wygrywa zawsze gracz rozpoczynający. Dlatego też gracze mają równą ilość wygranych gier. Inaczej sprawa ma się w przypadku gier probablistycznych. Wyniki graczy nie mają rozkładu 50/50.

Znaczenie odcięcia alpha nie wyróżnia się za bardzo na tle prób bez odcięcia alpha. Być może jest to źle dobrana wartość współczynnika alpha.

W przypadku czasu, to jest on bardzo zależny w głównej mierze od głebokości. Dopiero potem drugie skrzypce gra rodzaj (deterministyczny, probablistyczny).